In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs 
from urllib.request import urlopen
import urllib.request, urllib.parse, urllib.error
import requests
import ssl
import requests
import re
import pandas as pd
import math
import numpy as np
from numpy import dot
from numpy.linalg import norm
import os
import time
import pandas as pd
from PIL import Image
import pytesseract
import numpy as np
from pdf2image import convert_from_path, convert_from_bytes
from PyPDF2 import PdfReader
from PyPDF2 import PdfFileReader
from Crypto.Cipher import AES
import os
from datetime import datetime
from datetime import date
import re
import pandas as pd
import spacy
from spacy import displacy

In [ ]:
options = webdriver.ChromeOptions()
options.add_experimental_option('prefs', {
# "download.default_directory": "C:/Users/dhruv/Downloads", #Change default directory for downloads
"download.default_directory": r"D:\UTA\RA\Scrapped_Output",
"download.prompt_for_download": False, #To auto download the file
"download.directory_upgrade": True,
"plugins.always_open_pdf_externally": True, #It will not show PDF directly in chrome
"safebrowsing_for_trusted_sources_enabled": False,
"safebrowsing.enabled": False
})

In [ ]:
now_ = datetime.now().strftime("%H:%M:%S")
date_ = date.today().strftime("%b-%d-%Y")

log_file_path=r"D:\\UTA\\RA\\Scrapped_Output\\File_Generation_Log.txt"
log_file=open(log_file_path,mode='a',newline='\n')

In [ ]:
print(f"\n\n\nStart Date:{date_}\t\tStart time: {now_}",file=log_file)
browser = webdriver.Chrome(options = options)
print(f"\nInitial Web Driver Opened\n",file = log_file)
# df  = pd.read_excel(r'D:\UTA\RA\MW Entities Cusip_NoOS.xlsx')
# cusip = df.loc[51:,'cusip']
df  = pd.read_excel(r'D:\UTA\RA\Scrapped_Output\MW Entities Cusip_NoOS_TEST.xlsx')
cusip = df.loc[(df['Disclose_or_not'].isna()),'cusip']
# cusip = df.loc[82:,'cusip']
len(cusip)

In [ ]:
def move_into_view(os):
    browser.execute_script("return arguments[0].scrollIntoView({block: 'center', inline: 'nearest'})",os)

In [ ]:
def scrappy(CUSIP):
    print(f"\n\n\nStart Date:{date_}\t\tStart time: {now_}",file=log_file)
    print(f"\nCUSIP: {CUSIP}",file=log_file)
    browser.get('https://emma.msrb.org/Home/Index')
    actions = ActionChains(browser)

    try:
        search_bar = browser.find_element("xpath","//*[@id = 'quickSearchText']")
        search = search_bar.send_keys(CUSIP)
    except Exception as e:
        print(f"\nError: Issue with SEARCHING\n{e}\n\n",file=log_file)

    sleep(5)

    try:
        search_go = browser.find_element("xpath","//*[@id = 'quickSearchButton']")
        search_go.click()
    except Exception as e:
        print(f"Error: Could not SEARCH and GO;\n{e}\n\n",file=log_file)

    sleep(5)

    try:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        accept_go = browser.find_element("xpath","//input[@alternatetext='Accept']")
        accept_go.click()
    except Exception as e:
        print(f'Error: No Accept Button\n{e}\n\n',file=log_file)

    sleep(5)

    try:
        close_go = browser.find_element("xpath","//a[@class='closeOverlay']")
        close_go.click()
    except  Exception as e:
        print(f'Error: No Shadow Cross\n{e}\n\n',file=log_file)

    sleep(5)

    try:
        disc_docs_tab = browser.find_element("xpath","//*[@id = 'ui-id-4']")
        move_into_view(disc_docs_tab)
        sleep(3)
        disc_docs_tab.click()
    except  Exception as e:
        print(f"Error: Counld not find Disclosure Documents Tab\n{e}\n\n",file=log_file)

    sleep(3)

    try:
        os_cntnr = browser.find_element("xpath","//div[@id='officialStatementContainer']")
        if os_cntnr.find_element(By.TAG_NAME,'p').text == 'No OS has been submitted.':
            move_into_view(os_cntnr)
            print(f"{os_cntnr.find_element(By.TAG_NAME,'p').text}",file=log_file)
    except Exception as e:
        try:
            ls_os = browser.find_elements("xpath","//a[@ga-name='clickDisclosureDocuments/OS']")
            for os in ls_os:
                move_into_view(os)
                sleep(3)
                os.click()
                sleep(10)
        except Exception as e:
            print(f"Error: Could not download .pdf \n{e}\n\n",file=log_file)

    sleep(3)
    end_ = datetime.now().strftime("%H:%M:%S")
    end_date_ = date.today().strftime("%b-%d-%Y")
    print(f"\n\n\nEnd Date:{end_date_}\t\tEnd time: {end_}\nTime Take for completion->{datetime.strptime(end_, '%H:%M:%S') - datetime.strptime(now_, '%H:%M:%S')}",file=log_file)

    print(f"Restarting Web Driver\n\n",file=log_file)
    browser.get('https://emma.msrb.org/Home/Index')

In [ ]:
def pdf2img():
    for i in os.listdir('D:\\UTA\\RA\\Scrapped_Output'):
        if '.pdf' in i:
            images = convert_from_path(r"D:\\UTA\\RA\\Scrapped_Output\\"+i, poppler_path=r"D:\Poppler\poppler-23.11.0\Library\bin")
            for img in range(len(images)):
                images[img].save(r'D:\\UTA\\RA\\Scrapped_Output\\temp_images\\'+str(i.split('.pdf')[0])+'_IMG_'+ str(img) +'.jpg', 'JPEG')

In [ ]:
def keyW_search(CUSIP):
    p = []
    for i in os.listdir(r'D:\\UTA\\RA\\Scrapped_Output\\temp_images'):
        filename = r'D:\\UTA\\RA\\Scrapped_Output\\temp_images\\'+i
        if i.endswith(".jpg"):
            img1 = np.array(Image.open(filename))
            text = pytesseract.image_to_string(img1)
            paras = text.split('\n\n')
            for i in paras:
                if ("material weakness" in  i.lower()) :
                    p.append(i)
    if len(p)==0:
        df.loc[( df['cusip'] == CUSIP),'Disclose_or_not'] = 0
    else:
        df.loc[( df['cusip'] == CUSIP),'Disclose_or_not'] = 1

In [ ]:
def remove_pdf():
    for files in list(os.listdir(r"D:\\UTA\\RA\\Scrapped_Output\\")):
        if '.pdf' in  files:
            os.remove(r"D:\\UTA\\RA\\Scrapped_Output\\"+files)

In [ ]:
def remove_temp_imgs():
    for files in list(os.listdir(r"D:\\UTA\\RA\\Scrapped_Output\\temp_images\\")):
        os.remove(r"D:\\UTA\\RA\\Scrapped_Output\\temp_images\\"+files)

In [ ]:
for i in cusip:
    CUSIP = i
    remove_pdf()
    scrappy(CUSIP)
    pytesseract.pytesseract.tesseract_cmd = r'D:\Tesseract-OCR\tesseract.exe'
    pdf2img()
    keyW_search(CUSIP)
    remove_temp_imgs()
    df.to_excel(r"D:\UTA\RA\Scrapped_Output\MW Entities Cusip_NoOS_TEST.xlsx",index=False)
log_file.close()

In [ ]:
# df.loc[( df['Disclose_or_not'].isin(cusip)),:]
len(df.loc[(df['Disclose_or_not'].notna()),:])

In [ ]:
df.to_excel(r"D:\UTA\RA\Scrapped_Output\MW Entities Cusip_NoOS_TEST.xlsx",index=False)
log_file.close()